# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f741c259b20>
├── 'a' --> tensor([[ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812]])
└── 'x' --> <FastTreeValue 0x7f741c259a90>
    └── 'c' --> tensor([[-0.6123, -0.7129, -0.3391,  0.8624],
                        [ 0.2962, -0.3187,  0.9851,  0.0132],
                        [ 0.3767, -0.2043, -1.6620,  0.0178]])

In [4]:
t.a

tensor([[ 1.4787, -0.9441,  0.7472],
        [ 0.4769, -0.0566,  1.0812]])

In [5]:
%timeit t.a

70.8 ns ± 1.36 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f741c259b20>
├── 'a' --> tensor([[ 0.7680, -0.1635, -0.9780],
│                   [-0.3902, -0.1999,  0.2071]])
└── 'x' --> <FastTreeValue 0x7f741c259a90>
    └── 'c' --> tensor([[-0.6123, -0.7129, -0.3391,  0.8624],
                        [ 0.2962, -0.3187,  0.9851,  0.0132],
                        [ 0.3767, -0.2043, -1.6620,  0.0178]])

In [7]:
%timeit t.a = new_value

75.1 ns ± 1.7 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812]]),
    x: Batch(
           c: tensor([[-0.6123, -0.7129, -0.3391,  0.8624],
                      [ 0.2962, -0.3187,  0.9851,  0.0132],
                      [ 0.3767, -0.2043, -1.6620,  0.0178]]),
       ),
)

In [10]:
b.a

tensor([[ 1.4787, -0.9441,  0.7472],
        [ 0.4769, -0.0566,  1.0812]])

In [11]:
%timeit b.a

66.1 ns ± 1.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4684,  0.3782, -0.1944],
               [ 0.6940, -0.3559, -1.1629]]),
    x: Batch(
           c: tensor([[-0.6123, -0.7129, -0.3391,  0.8624],
                      [ 0.2962, -0.3187,  0.9851,  0.0132],
                      [ 0.3767, -0.2043, -1.6620,  0.0178]]),
       ),
)

In [13]:
%timeit b.a = new_value

599 ns ± 6.46 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.02 µs ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.9 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

264 µs ± 5.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

254 µs ± 4.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7377c0dcd0>
├── 'a' --> tensor([[[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]],
│           
│                   [[ 1.4787, -0.9441,  0.7472],
│                    [ 0.4769, -0.0566,  1.0812]]])
└── 'x' --> <FastTreeValue 0x7f741c2b8d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.3 µs ± 626 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7370f28f70>
├── 'a' --> tensor([[ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812],
│                   [ 1.4787, -0.9441,  0.7472],
│                   [ 0.4769, -0.0566,  1.0812]])
└── 'x' --> <FastTreeValue 0x7f741c225550>
    └── 'c' --> tensor([[-0.6123, -0.7129, -0.3391,  0.8624],
                        [ 0.2962, -0.3187,  0.9851,  0.0132],
                 

In [23]:
%timeit t_cat(trees)

41.5 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

82.7 µs ± 4.45 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]],
       
               [[ 1.4787, -0.9441,  0.7472],
                [ 0.4769, -0.0566,  1.0812]]]),
    x: Batch(
           c: tensor([[[-0.6123, -0.7129, -0.3391,  0.8624],
                       [ 0.2962, -0.3187,  0.9851,  0.0132],
                       [ 0.3767, -0.2043, -1.6620,  0.0178]],
         

In [26]:
%timeit Batch.stack(batches)

101 µs ± 7.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812],
               [ 1.4787, -0.9441,  0.7472],
               [ 0.4769, -0.0566,  1.0812]]),
    x: Batch(
           c: tensor([[-0.6123, -0.7129, -0.3391,  0.8624],
                      [ 0.2962, -0.3187,  0.9851,  0.0132],
                      [ 0.3767, -0.2043, -1.6620,  0.0178],
                      [-0.6123, -0.7129, -0.3391,  0.8624],
                      [ 0.2962, -

In [28]:
%timeit Batch.cat(batches)

204 µs ± 9.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

610 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
